In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_excel("tennis_set.xlsx")
train_df

,Day,Outlook,Temperature,Humidity,Wind,PlayTennis
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [3]:
def prior(data_a, column_a, value_a):
    res = len(data_a[data_a[column_a] == value_a]) / (len(data_a)+0.0001)
    #print ("prior ",  column_a, value_a, res)
    return res

In [4]:
def likelyhood(data_only_a, column_b, value_b):
    res = len(data_only_a[data_only_a[column_b] == value_b]) / (len(data_only_a)+0.0001)
    #print ("likelyhood ", column_b, value_b, res)
    return res

In [5]:
def evidence(data_a, column_a, column_b, value_b):
    P_B = 0
    for a_i in data_a[column_a].unique():
        P_B += prior(data_a, column_a, a_i) * likelyhood(data_a[data_a[column_a] == a_i], column_b, value_b)
    return P_B

In [6]:
def probability(data_a, column_a, value, column_b, hypothes):
    _prior = prior(data_a, column_a, value)
    _likelyhood = likelyhood(data_a[data_a[column_a] == value], column_b, hypothes)
    _evidence = evidence(data_a, column_a, column_b, hypothes)
    #P(A_value|B_hypothes)
    return  (_prior*_likelyhood /_evidence)

In [7]:
categories = list(train_df)
categories

['Day', 'Outlook', 'Temperature', 'Humidity', 'Wind', 'PlayTennis']

In [8]:
for cat in categories:
    print(pd.unique(train_df[cat]))

['D1' 'D2' 'D3' 'D4' 'D5' 'D6' 'D7' 'D8' 'D9' 'D10' 'D11' 'D12' 'D13'
 'D14']
['Sunny' 'Overcast' 'Rain']
['Hot' 'Mild' 'Cool']
['High' 'Normal']
['Weak' 'Strong']
['No' 'Yes']


In [9]:
train_df

,Day,Outlook,Temperature,Humidity,Wind,PlayTennis
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


In [10]:
B_column = 'Outlook'
B_hypothes = 'Sunny'
A_column = 'PlayTennis'
A_value = 'No'
print(f"P({A_column}={A_value}|{B_column}={B_hypothes}) =", probability(train_df, A_column, A_value, B_column, B_hypothes))

P(PlayTennis=No|Outlook=Sunny) = 0.5999978666979551


In [11]:
def print_P(data_a, column_a, value_a, column_b, value_b):
    print(f"P({column_a}={value_a}|{column_b}={value_b}) =", probability(data_a, column_a, value_a, column_b, value_b))
    return

In [12]:
print_P(train_df,'PlayTennis', 'No','Outlook',  'Sunny' )

P(PlayTennis=No|Outlook=Sunny) = 0.5999978666979551


In [13]:
print_P(train_df,'Outlook',  'Sunny', 'PlayTennis', 'No')

P(Outlook=Sunny|PlayTennis=No) = 0.6000000000000001


In [14]:
outlook = ['Sunny', 'Overcast', 'Rain']
tempreture = ['Hot', 'Mild', 'Cool']
humidity = ['High', 'Normal']
wind= ['Weak', 'Strong']
result = ['No', 'Yes']

for o in outlook:
    for r in  result:
        print(o,r)
        print_P(train_df,'Outlook',  o, 'PlayTennis', r)

Sunny No
P(Outlook=Sunny|PlayTennis=No) = 0.6000000000000001
Sunny Yes
P(Outlook=Sunny|PlayTennis=Yes) = 0.22222271603813468
Overcast No
P(Outlook=Overcast|PlayTennis=No) = 0.0
Overcast Yes
P(Outlook=Overcast|PlayTennis=Yes) = 0.4444432099046633
Rain No
P(Outlook=Rain|PlayTennis=No) = 0.4
Rain Yes
P(Outlook=Rain|PlayTennis=Yes) = 0.33333407405720206


In [15]:
def prob_yes(data_a, value_a, value_b, value_c, value_d):
    return (
    probability(data_a, 'Outlook', value_a, 'PlayTennis', 'Yes') *
    probability(data_a, 'Temperature', value_b, 'PlayTennis', 'Yes') *
    probability(data_a, 'Humidity', value_c, 'PlayTennis', 'Yes') *
    probability(data_a, 'Wind', value_d, 'PlayTennis', 'Yes') *
    prior(data_a,  'PlayTennis', 'Yes')
    )
def prob_no(data_a, value_a, value_b, value_c, value_d):
    return (
    probability(data_a, 'Outlook', value_a, 'PlayTennis', 'No') *
    probability(data_a, 'Temperature', value_b, 'PlayTennis', 'No') *
    probability(data_a, 'Humidity', value_c, 'PlayTennis', 'No') *
    probability(data_a, 'Wind', value_d, 'PlayTennis', 'No') *
    prior(data_a,  'PlayTennis', 'No')
    )
def proba_full(data_a, value_a, value_b, value_c, value_d):
    return (
    prob_yes(data_a, value_a, value_b, value_c, value_d) + 
    prob_no(data_a, value_a, value_b, value_c, value_d)
    ) 

In [16]:
def Bayes_fit(data_a, value_a, value_b, value_c, value_d, hypothes):
    return (
    probability(data_a, 'Outlook', value_a, 'PlayTennis', hypothes) *
    probability(data_a, 'Temperature', value_b, 'PlayTennis', hypothes) *
    probability(data_a, 'Humidity', value_c, 'PlayTennis', hypothes) *
    probability(data_a, 'Wind', value_d, 'PlayTennis', hypothes) *
    prior(data_a,  'PlayTennis', hypothes) / 
    proba_full (data_a, value_a, value_b, value_c, value_d)
    )   

In [17]:
Bayes_fit(train_df, 'Sunny', 'Cool', 'High', 'Strong', 'Yes')

0.2045827719261247

In [18]:
Bayes_fit(train_df, 'Sunny', 'Cool', 'High', 'Strong', 'No')

0.7954172280738753

In [19]:
def TestBayes(value_a, value_b, value_c, value_d):
    yes= Bayes_fit(train_df, value_a, value_b, value_c, value_d, 'Yes')
    no= Bayes_fit(train_df, value_a, value_b, value_c, value_d, 'No')
    print (" %s %s %s %s Yes %f" % (value_a, value_b, value_c, value_d, yes))
    print (" %s %s %s %s No  %f" % (value_a, value_b, value_c, value_d, no))
    return 

In [21]:
TestBayes('Sunny', 'Cool', 'Normal', 'Strong')

 Sunny Cool Normal Strong Yes 0.672948
 Sunny Cool Normal Strong No  0.327052
